In [28]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d

Create grids for next period's capital holdings

In [4]:
def create_grids(n, theta):

    zero_grid=np.linspace(0, 1, n).reshape(-1,1)

    k_grid=np.zeros(shape=(n,1))

    k_low  = 0.01

    k_high  = 2
    
    #for i in range(len(new_grid)):
        #k_grid[i] =  low_bound + ( up_bound - low_bound ) * (new_grid[i] ** 1)

    for i in range(len(zero_grid)):
        k_grid[i] =  (0.25 * k_low) + ( (1.5 * k_high) - (0.25 * k_low) ) * (zero_grid[i] ** theta)

    return k_grid 

define various utility functions

In [5]:
def u(c, gamma):
    if gamma == 1:
        return np.log(c)
    else:
        return ((c ** (1 - gamma))/( 1 - gamma))

In [6]:
def u_ces(c,gamma, beta):
    return (1 - beta) * (c) ** (1-gamma)

define the production function

In [7]:
def f(A, alpha, delta, k):
    return A * (k ** alpha) + (1 - delta) * k

Now, create a function to linearly interpolate each column of the Value matrix, so that we can calculate the derivate at each point on the capital grid.

In [34]:
def interpolate_column(grid, column):
    interpolator = interp1d(np.squeeze(grid), column, kind = 'linear', fill_value = 'extrapolate')
    return interpolator

Let's now begin with the endogenous grid method

In [35]:
k_grid = create_grids(101, 3)
# V = np.zeros((101, 15))

# for i in range(len(V[0])):
#    for j in range(len(V[0])):
#        V[i,j]=(0.01*(i+1))*(j+1)

i_vals = np.arange(1,102)
j_vals = np.arange(1,16)

V = (0.01 * (k_grid)) * j_vals  

# print(V.shape)

#for j in range(len(V[1])):
#   print(f'column {j}')
#  print(V[:,j])

#for j in range(len(V[1])):
#
#    fig, ax = plt.subplots(figsize=(10, 6))
#
#    for j in range(len(V[1])):
#        ax.plot(k_grid, V[:, j], label=f'TFP Shock {j}')

#    ax.set_title('Value Function')
#    ax.set_xlabel('Capital Stock')
#    ax.set_ylabel('Value')
#    ax.legend()
#    plt.show()


In [32]:
def egm(n_k, n_z, theta, rho, sigma, mu, A, delta, max_iter ):
    # First, discretise an AR1 process using Rouwenhorst's method
    mc = qe.markov.approximation.rouwenhorst(n_z, rho, sigma, mu)
    z, Pi = mc.state_values, mc.P

    # Create capital and shock grids
    z_grid = z
    k_grid = create_grids(n_k, theta)

    # Initialise value and policy functions
    j_vals = np.arange(1,16)

    V = (0.01 * (k_grid)) * j_vals  

    G_kp = np.zeros((n_k, n_z))

    G_kp_old = np.copy(G_kp)

    it = 0

    for it in range(max_iter):

        interpolators = [interpolate_column(k_grid, V[:,j]) for j in range(V[1])]

        


    it+=1
        

